In [3]:
import os 
import pandas as pd
import numpy as np

os.chdir('S:/Spring/NLP')
os.listdir()

['Assignment 1 - Facebook Post Classification.ipynb',
 'FB_posts_labeled.txt',
 'FB_posts_unlabeled.txt',
 'Text Preprocessing and Representation Learning.html']

In [4]:
df_post_label = pd.read_csv("FB_posts_labeled.txt",header=0,delimiter="\t")
df_test = pd.read_csv('FB_posts_unlabeled.txt',header=0,sep = '\t')

df_test.shape

(2039, 2)

In [5]:
from sklearn.preprocessing import LabelEncoder

label_cols = ['Appreciation', 'Complaint', 'Feedback']
df_post_label[label_cols] = df_post_label[label_cols].apply(pd.to_numeric, axis = 1, errors = 'coerce')

df_post_label['label']= df_post_label[['Appreciation', 'Complaint', 'Feedback']].idxmax(axis = 1)
df_post_label.head()



le = LabelEncoder()
df_post_label['label_encoded'] = le.fit_transform(df_post_label['label'])
df_post_label.head()

,postId,message,Appreciation,Complaint,Feedback,label,label_encoded
0,126016648090_10150802142013091,Great ! ;),1,0,0,Appreciation,0
1,108381603303_10151136215833304,YUM! YUM!,1,0,0,Appreciation,0
2,108381603303_3913438087739,Yummm :)),1,0,0,Appreciation,0
3,110455108974424_343049739048292,sweet,1,0,0,Appreciation,0
4,110455108974424_350358541650745,nice,1,0,0,Appreciation,0


In [5]:
label_dict = {label: index for index, label in enumerate(set(df_post_label['label_encoded']))}
label_dict

{0: 0, 1: 1, 2: 2}

In [20]:
import re
df_post_label['message'] = df_post_label['message'].apply(lambda x: x.lower())
df_post_label['message'] = df_post_label['message'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df_post_label.head()

,postId,message,Appreciation,Complaint,Feedback,label,label_encoded
0,126016648090_10150802142013091,great,1,0,0,Appreciation,0
1,108381603303_10151136215833304,yum yum,1,0,0,Appreciation,0
2,108381603303_3913438087739,yummm,1,0,0,Appreciation,0
3,110455108974424_343049739048292,sweet,1,0,0,Appreciation,0
4,110455108974424_350358541650745,nice,1,0,0,Appreciation,0


In [30]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_fatures = 2000
max_len = 130
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_post_label['message'].values)
X = tokenizer.texts_to_sequences(df_post_label['message'].values)
X = pad_sequences(X, maxlen = max_len)

In [31]:
X.shape

(7961, 130)

In [9]:
from sklearn.model_selection import train_test_split

Y = pd.get_dummies(df_post_label['label_encoded']).values
Y
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_test.shape,Y_test.shape)

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [ ]:
Y_train

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 200
lstm_out = 64
model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 130, 200)          400000    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 130, 200)         0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 64)                67840     
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 468,035
Trainable params: 468,035
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
# Here we train the model
batch_size = 64
hist = model.fit(X, Y, epochs = 2, batch_size=batch_size, verbose = 2)

Epoch 1/2
125/125 - 59s - loss: 0.8133 - accuracy: 0.6364 - 59s/epoch - 472ms/step
Epoch 2/2
125/125 - 57s - loss: 0.4917 - accuracy: 0.8157 - 57s/epoch - 453ms/step


In [35]:


df_test['message'] = df_test['message'].apply(lambda x: x.lower())
df_test['message'] = df_test['message'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

df_test


,postId,message
0,108381603303_10151119973393304,love it to
1,115568331790246_371841206162956,nice
2,115568331790246_515044031842672,congrats
3,147285781446_10151010892176447,awesome
4,159616034235_10150639103634236,award
...,...,...
2034,179590995428478_422375854483323,you guys are terrible holding goverment checks...
2035,125472670805257_525103854175468,as i platinum elite member of delta and a loya...
2036,179590995428478_377568608964048,really
2037,179590995428478_341070505947192,horrible decision


In [36]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_test['message'].values)
X_test = tokenizer.texts_to_sequences(df_test['message'].values)
X_test = pad_sequences(X_test, maxlen = max_len)
X_test

array([[   0,    0,    0, ...,   84,   13,    2],
       [   0,    0,    0, ...,    0,    0,  349],
       [   0,    0,    0, ...,    0,    0, 1928],
       ...,
       [   0,    0,    0, ...,    0,    0,   73],
       [   0,    0,    0, ...,    0,  460,  604],
       [ 168,  307,    6, ...,    2,  101,   52]])

In [46]:
X_test.shape

(2039, 130)

In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 130, 200)          400000    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 130, 200)         0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 64)                67840     
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 468,035
Trainable params: 468,035
Non-trainable params: 0
_________________________________________________________________


In [25]:
X_test.shape

(2039, 631)

In [48]:
predict_raw = model.predict(X_test)
predict_raw[0]

64/64 [==============================] - 3s 36ms/step


array([0.10398594, 0.25258347, 0.6434306 ], dtype=float32)

In [49]:
y_predict = np.argmax(predict_raw, axis = 1)
out = pd.Series(y_predict)
df_test['category'] = out
df_test

,postId,message,category
0,108381603303_10151119973393304,love it to,2
1,115568331790246_371841206162956,nice,1
2,115568331790246_515044031842672,congrats,1
3,147285781446_10151010892176447,awesome,0
4,159616034235_10150639103634236,award,1
...,...,...,...
2034,179590995428478_422375854483323,you guys are terrible holding goverment checks...,1
2035,125472670805257_525103854175468,as i platinum elite member of delta and a loya...,1
2036,179590995428478_377568608964048,really,1
2037,179590995428478_341070505947192,horrible decision,1


In [50]:
df_result = pd.get_dummies(df_test['category'], columns = ['Appreciation_pred','Complaint_pred', 'Feedback_pred'])
df_result.columns = ['Appreciation_pred','Complaint_pred', 'Feedback_pred']
output = pd.concat([df_test, df_result], axis = 1)

output.drop(['message','category'], axis=1, inplace = True)
output.to_csv('predictions.csv')

In [51]:
output

,postId,Appreciation_pred,Complaint_pred,Feedback_pred
0,108381603303_10151119973393304,0,0,1
1,115568331790246_371841206162956,0,1,0
2,115568331790246_515044031842672,0,1,0
3,147285781446_10151010892176447,1,0,0
4,159616034235_10150639103634236,0,1,0
...,...,...,...,...
2034,179590995428478_422375854483323,0,1,0
2035,125472670805257_525103854175468,0,1,0
2036,179590995428478_377568608964048,0,1,0
2037,179590995428478_341070505947192,0,1,0
